eTL에 첨부된 `train_data.csv` 파일을 Colab에 업로드하여 사용한다.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
data = pd.read_csv('train_data.csv', index_col=0)
data

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...
20437,3,0,3,0,1,RT @semper_fiibitch: Farra from teen mom has 2...
20438,3,0,0,3,2,RT @sepinwall: Day 2 of FXX&#8217;s Simpsons m...
20439,3,0,3,0,1,"RT @shabbashone ""@ThoughtsOfRandy: I almost ra..."
20440,3,0,2,1,1,RT @shaelynspacyyy: &#8220;@UglyAssDerrick: @s...


In [ ]:
y = data['class'].copy()

In [ ]:
stop_words = set(stopwords.words("english"))
stop_words.add('rt')
stop_words.remove('not')
lemmatizer = WordNetLemmatizer()
def preprocess(text):
    text = re.sub('"', "", text)
    text = re.sub('@[\w\-]+', ' ',text) #계정 이름 지우기
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|' '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)  #url링크 지우기
    text = text.lower()
    text = re.sub("hm+", "", text) #removing variants of hmmm
    text = re.sub("[^a-z]+", " ", text) #숫자와 특수문자 지우기
    text = text.split()
    text = [word for word in text if not word in stop_words] #stopwords 지우기
    #text = [d[word] if word in d else word for word in text]
    text = [lemmatizer.lemmatize(token) for token in text]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = " ".join(text)
    return text

corpus = data['tweet'].apply(preprocess)

In [ ]:
#하이퍼 패러미터 향상을 못하겠다...
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(n)
x = vectorizer.fit_transform(corpus).toarray()
print(x.shape)

(20000, 17816)


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.1)

In [ ]:
#로지스틱회귀함수

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l2",C=0.01,verbose=1, random_state=0, max_iter=2000))
x_ = select.fit_transform(x,y)
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(x_, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.1s finished


In [ ]:
model.fit(x_train, y_train)

LinearSVC(C=0.01, class_weight='balanced')

In [ ]:
true = y_dev
pred = model.predict(x_dev)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(true, pred))
print(confusion_matrix(true, pred))
print(classification_report(true, pred))

0.8665
[[  31   87   15]
 [  22 1472   36]
 [   2  105  230]]
              precision    recall  f1-score   support

           0       0.56      0.23      0.33       133
           1       0.88      0.96      0.92      1530
           2       0.82      0.68      0.74       337

    accuracy                           0.87      2000
   macro avg       0.76      0.63      0.67      2000
weighted avg       0.85      0.87      0.85      2000



In [ ]:
#svm

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
svm_model = LinearSVC(class_weight='balanced',multi_class='crammer_singer',max_iter = -1).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
svm_model_predict = svm_model.predict(x_dev)
svm_report = classification_report(y_dev, svm_model_predict )

In [ ]:
print(svm_report)

              precision    recall  f1-score   support

           0       0.35      0.53      0.42       121
           1       0.96      0.88      0.92      1547
           2       0.79      0.96      0.86       332

    accuracy                           0.87      2000
   macro avg       0.70      0.79      0.73      2000
weighted avg       0.90      0.87      0.88      2000



In [ ]:
#svm2

In [ ]:
svm_model_2 = LinearSVC(class_weight='balanced',C=1, penalty='l2', max_iter=1500,loss='squared_hinge',multi_class='ovr').fit(x_train, y_train)

In [ ]:
svm_model_predict_2 = svm_model_2.predict(x_dev)
svm_report_2 = classification_report(y_dev, svm_model_predict_2 )

In [ ]:
print(svm_report_2)

              precision    recall  f1-score   support

           0       0.43      0.36      0.39       121
           1       0.95      0.93      0.94      1547
           2       0.81      0.94      0.87       332

    accuracy                           0.89      2000
   macro avg       0.73      0.74      0.73      2000
weighted avg       0.89      0.89      0.89      2000



In [ ]:
#교차검증

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
cross_val_score(model.fit(x_train, y_train), x_train, y_train, scoring=make_scorer(f1_score, average='weighted', labels=[0]), cv=10)

array([0.30379747, 0.30065359, 0.34782609, 0.26415094, 0.375     ,
       0.36129032, 0.31168831, 0.40490798, 0.26506024, 0.30379747])

In [ ]:
cross_val_score(LinearSVC(class_weight='balanced',C=1, penalty='l2', max_iter=1500,loss='squared_hinge',multi_class='ovr').fit(x_train, y_train), x_train, y_train, scoring=make_scorer(f1_score, average='weighted', labels=[0]), cv=10)

array([0.39814815, 0.3930131 , 0.45070423, 0.36097561, 0.34513274,
       0.35185185, 0.34482759, 0.42918455, 0.32535885, 0.32142857])